## 파이썬 완정정복 클래스 내부 메소드 생성 규칙 및 정보은닉 처리 알아보기


In [1]:
import sys

In [2]:
sys.version_info

sys.version_info(major=3, minor=6, micro=6, releaselevel='final', serial=0)

### 클래스 네임스페이스 알아보기

In [3]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    def getPerson(self):
        return self.name, self.age
    


#### 사용자 클래스와 object 클래스 네임스페이스 비교

In [4]:
p = set(Person.__dict__)    
o = set(object.__dict__)

print(p - o)


{'__module__', '__dict__', '__weakref__', 'getPerson'}


####  공통 요소 확인 

In [5]:
print(p & o)

{'__doc__', '__init__'}


### 인스턴스 네임스페이스 확인 

In [6]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    def getPerson(self):
        return self.name, self.age


#### 인스턴스 네임스페이스는 기본적으로 초기화 내부의 속성을 가진다.

In [7]:
p = Person('dahl', 50)
print(p)
print(p.__dict__)

{'name': 'dahl', 'age': 50}


### 메소드 정의할 때 내부 속성 확인 

In [8]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    def getPerson(self):
        var_mt = "method attribute"
        print("getPerson", locals())
        return self.name, self.age
    

#### 정의한 것은 메소드가 아닌 함수이므로 내부 속성을 지역변수

In [9]:
p = Person("dahl", 50)
print(p)
print(p.__dict__)
print(p.getPerson())

{'name': 'dahl', 'age': 50}
getPerson {'var_mt': 'method attribute', 'self': <__main__.Person object at 0x00000000065BF2B0>}
('dahl', 50)


### 보호 속성 확인하기 

      관행상 보호 속성이므로 실질적으로는 접근이 가능하다

#### 초기화 내부에 보호속성 메소드를 호출해서 보호속성이 인스턴스에 들어가도록 한다

In [10]:
class Protected:
    def __init__(self, name, age):
        self._set(name, age)
        
    def _set(self, name, age):
        self._name = name
        self._age = age
        
    def getname(self):
        return self._name
    
    def getage(self):
        return self._age

#### 인스턴스에 보호속성이 생성되어 들어간다

In [11]:
p = Protected("정찬혁", 31)

print(p.__dict__)

{'_name': '정찬혁', '_age': 31}


#### 일반 메소드들도 내부에서 보호속성으로 처리하도록 한다

In [12]:
print(p.getname())
print(p.getage())

정찬혁
31


#### 직접 접근도 가능하다

In [13]:
print(p._name)
print(p._age)

정찬혁
31


### 맹글링으로 속성 보호하기

In [14]:
class Mangling:
    def __init__(self, name, age):
        self.__set(name, age)
        
    def __set(self, name, age):
        self.__name = name
        self.__age = age
        
    def getname(self):
        return self.__name
    
    def getage(self):
        return self.__age

In [15]:
p = Mangling("정찬혁", 31)
print(p.__dict__)

{'_Mangling__name': '정찬혁', '_Mangling__age': 31}


In [16]:
print(p.getname())
print(p.getage())

정찬혁
31


#### 실제 접근하면 예외가 발생한다.

In [17]:
try: 
    p.__name
    
except Exception as e :
    print(e)


'Mangling' object has no attribute '__name'


#### 전체 이름으로 접근한다.

In [18]:
print(p._Mangling__name)
print(p._Mangling__age)

정찬혁
31


#### 맹글링으로 정의된 내부 확인하기 

In [19]:
import pprint

pprint.pprint(Mangling.__dict__)

mappingproxy({'_Mangling__set': <function Mangling.__set at 0x00000000065C2730>,
              '__dict__': <attribute '__dict__' of 'Mangling' objects>,
              '__doc__': None,
              '__init__': <function Mangling.__init__ at 0x00000000065C27B8>,
              '__module__': '__main__',
              '__weakref__': <attribute '__weakref__' of 'Mangling' objects>,
              'getage': <function Mangling.getage at 0x00000000065C2620>,
              'getname': <function Mangling.getname at 0x00000000065C26A8>})


#### 인스턴스에서 맹글링 메소드 접근 

In [20]:
try :
    p.__set("맹글링", 55)
except Exception as e :
    print(e)

'Mangling' object has no attribute '__set'


#### 맹글링 풀 이름으로 메소드 접근하면 실행 가능

In [21]:
p._Mangling__set("맹글링", 55)

In [22]:
p.__dict__

{'_Mangling__age': 55, '_Mangling__name': '맹글링'}

### 메소드는 오버로딩 불가

#### 클래스 내의 함수가 네임스페이스로 갈 때 

     이름이 동일하므로 딕셔너리에 첫번째가 할당되고 이후에 두번째의 함수 객체가 값으로 들어간다.

In [23]:
class X:
    def add(self):
        pass
    print("add 1 ", add)
    def add(self, x):
        return x
    print("add 2 ", add)

add 1  <function X.add at 0x00000000065C2268>
add 2  <function X.add at 0x00000000065C20D0>


In [24]:
x = X()
print(X.__dict__)
print(x.add(5))
print(type(x.add))

{'__module__': '__main__', 'add': <function X.add at 0x00000000065C20D0>, '__dict__': <attribute '__dict__' of 'X' objects>, '__weakref__': <attribute '__weakref__' of 'X' objects>, '__doc__': None}
5
<class 'method'>


#### 첫번째 함수를 가진 메소드는 없는 것을 알 수 있다.

In [25]:
try :
    x.add()
except Exception as e :
    print(e)

add() missing 1 required positional argument: 'x'


### 인스턴스 메소드의 self 이해하기

#### 클래스 정의할 때 내부의 함수에 self 자리를 없애고 가변인자로 받는다

       대신 인스턴스 위치에 가변인자의 첫번째 위치를 가져같다.
       클래스 내부에 정의된 인스턴스 메소드는 첫번째 자리가 이름과 상관없이 인스턴스가 들어가는 것을 알 수 있다.

In [26]:
class Init:
    def __init__(*args):
        print(args)
        args[0].name = args[1]
        
    def getName(*args):
        return args[0].name
    

####  실제 인스턴스를 만들고 실행해도 잘 처리가 된다

In [27]:
c = Init('dahl')

print(c)

print(c.getName())

(<__main__.Init object at 0x00000000065B6978>, 'dahl')
dahl


### 실제 self 정보를 확인하기 

In [28]:
class A:
    def __init__(self, name):
        print("__self__", self.__init__.__self__)
        self.name = name


In [29]:
a = A('dahl')


__self__ <__main__.A object at 0x00000000065BFE48>


#### 메소드 내부에 __self__, __func__ 이 생성되어 있다. 

In [30]:
print(dir(a.__init__))
print(a.__init__.__class__)
print(a.__init__.__func__)
print(a.__init__.__self__)

['__call__', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__func__', '__ge__', '__get__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__self__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']
<class 'method'>
<function A.__init__ at 0x00000000065BABF8>


### 특정 속성을 정의하고 그 속성으로만 처리해 보기 


      slots 처러 처리가 가능한 이유는 실제 인스턴스 속성에 들어가는 것은 항상 런타임에 결정되는 것을 알 수 있다.
      

#### 클래스 속성에 들어갈 속성 이름만 정의 

In [31]:
class Self:
    attr = ("name", 'age')
    def __init__(*args):
        print("__self__ argument ", args[0])
        print("__self__ attribute ", args[0].__init__.__self__)
        
        for i in range(1, len(args)):
            args[0].__dict__[Self.attr[i-1]] = args[i]
            
    def get(*args):
        print("__self__ attribute ", args[0].get.__self__)
        return args[0].name, args[0].age

#### 객체를 생성하면 실제 정의된 2개으 속성으로 들어가 있는 것을 볼 수 있다.

In [32]:
s = Self('Dahl', 22)
print(s.__dict__)
print(s)

__self__ argument  <__main__.Self object at 0x00000000065E9828>
__self__ attribute  <__main__.Self object at 0x00000000065E9828>
{'name': 'Dahl', 'age': 22}


### 클래스 메소드에서 cls 알아보기

In [33]:
class Cls:
    name = 'dahl'

    @classmethod
    def getName(*args):
        print("class method ", args[0])
        return args[0].name
    

In [34]:
print(Cls.getName())
print(Cls)

class method  <class '__main__.Cls'>
dahl
<class '__main__.Cls'>


In [35]:
dir(Cls.getName)

['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__func__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__self__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

#### 클래스 메소드 내에도 실제 __self__, __func__이 존재

In [36]:
Cls.getName.__self__

__main__.Cls

In [37]:
Cls.getName.__func__

<function __main__.Cls.getName>

### cls 를 사용하는 이유는 관행

In [38]:
class ClsKlass:
    name = "클래스"
    def __init__(self, name):
        self.name = name
        
    @classmethod
    def getname(cls):
        return cls.name

In [39]:
a = ClsKlass("인스턴스")

print(ClsKlass.getname)

<bound method ClsKlass.getname of <class '__main__.ClsKlass'>>


#### 함수와 메소드의 차이를 비교

    인스턴스메소드는 런타임 시에 결정되므로 클래스에서 부르면 함수로 처리

In [40]:
s = set(dir(ClsKlass.__init__))
c = set(dir(ClsKlass.getname))

print(c - s)

{'__func__', '__self__'}


#### 클래스 메소드는 로딩할 때 만들어지므로 실제 함수는 __func__에만 존재한다

In [41]:
print(ClsKlass)
print(ClsKlass.getname.__self__)
print(id(ClsKlass.getname))
print(id(ClsKlass.getname.__func__))

<class '__main__.ClsKlass'>
<class '__main__.ClsKlass'>
103491272
106702096


### 클래스 메소드를 이용해서 초기화하기

In [42]:
class B:
    @classmethod
    def init(cls, name):
        self = B()
        setattr(self, 'name', name)
        print('cls bound', B.init.__self__)
        return self


In [43]:
b = B.init('name')
print(b.name)

cls bound <class '__main__.B'>
name


In [44]:
b.__dict__

{'name': 'name'}

### 정적 메소드로 초기화 하기

In [45]:
class C:
    @staticmethod
    def init(name):
        self = C()
        setattr(self, 'name', name)
        print("static bound ", getattr(C.init, "__self__", None))
        return self
    

In [46]:
c = C.init('Park')
print(c)

static bound  None


#### 정적메소드 확인 

In [47]:
print(dir(C.init))

['__annotations__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__globals__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']


#### 정적메소드는 일반적인 함수와 동일하지만 클래스나 인스턴스로 처리가 가능한 형태로 변경

In [48]:
C.init

<function __main__.C.init>

In [49]:
try :
    C.init.__self__
except Exception as e :
    print(e)

'function' object has no attribute '__self__'


In [50]:
try :
    C.init.__func__
except Exception as e :
    print(e)

'function' object has no attribute '__func__'


### 클래스 내부에 일반 함수 정의 

In [51]:
class Ins:
    def __init__(self, name):
        self.name = name
        
    def __call__(self):
        return self.name
    
    def create(name):
        a = Ins(name)
        return a
    


In [52]:
a = Ins("Dahl")()
print(a)

Dahl


#### 클래스 내부에 함수를 정의해도 처리결과가 같다.

In [53]:
b = Ins.create("Moon")
print(b)

In [54]:
b()

'Moon'

### 메소드 체인 처리하기

      메소드 반환값에 객체 인스턴스를 리턴하도록 로직을 넣어서 처리한다.
      
      그러면 인스턴스의 객체에 변경된 것을 로직에서 반영하고 자기 객체만 매번 전달한다.
      
      단 무한순환이 되지 않도록 마지막은 결과를 종료하도록 반환값을 객체로 전달하지 않는다.
      

In [55]:
class Person:

    def name(self, name):
        self.name = name
        return self

    def age(self, value):
        self.age = value
        return self

    def introduce(self):
        print("Hello, my name is", self.name, "and I am", self.age, "years old.")


####  인스턴스가 연속적으로 전달되므로 연속해서 처리 가능

In [56]:

person = Person()
#객체의 메소드를 연속적으로 호출하여 처리

person.name("Peter").age(21).introduce()

Hello, my name is Peter and I am 21 years old.


#### 인스턴스를 생성하면서 바로 메소드 체인도 처리가 가능

In [57]:
Person().name("조현웅").age(21).introduce()

Hello, my name is 조현웅 and I am 21 years old.


###  다른 클래스의 인스턴스를 받아서 메소드 체인 처리하기 

#### 첫번째 클래스에서는 다른 클래스를 속성으로 생성 

In [58]:
class A:
    def __init__(self):
        print("a")
        self.b = B()
        
    

#### 두번째 클래스에서 인스턴스를 만들때 다른 클래스를 속성으로 받고 메소드 처리시에도 리턴을 다른 인스턴스를 넘긴다

In [59]:
class B:
    def __init__(self):
        self.c = C()
        print('b')
        
    def bbb(self):
        print("B instance method")
        return self.c
    

#### 마지막을 처리하는 클래스를 정의한다

In [60]:
class C:
    def __init__(self):
        print('c')
        
    def ccc(self):
        print("C instance method")
        

#### 인스턴스를 만들고 속성에 있는 인스턴스를 호출한 후에 메소드 체인처리

In [61]:
a = A()

a.b.bbb().ccc()

a
c
b
B instance method
C instance method


###  메소드 오버로딩 알아보기

#### 상속을 받고  add 연산자를 클래스에 정의

In [62]:
class INT(int):
    def __init__(self, value):
        self.value = value
        
    def __add__(self, other):
        s = INT(0)
        s.value = self.value + other.value
        return s
    
    def __str__(self):
        return str(self.value)


In [63]:
c = INT(10)    
d = INT(20)
print(id(c))
print(id(d))


106900296
106900360


In [64]:
e = c + d
print(id(e))
print(e)
print(e.value)

106773768
30
30


#### int 클래스를 상속 받아서 모든 연산을 처리가능

In [65]:
print(d - c)
print(d*c)

10
200


### 상속없이 연산자 대치하기 

In [66]:
class INT_:
    def __init__(self, value):
        self.value = value
        
    def __add__(self, other):
        return self.value +  other.value
    
    def __str__(self):
        return str(self)
    

In [67]:
c = INT_(10)    
d = INT_(20)

e = c + d

print(e)

30


#### 상속을 받지 않아서 연산자 오버로딩한 것만 처리가 됨

In [68]:
try :
    
    print(d - c)
    print(d*c)
except Exception as e :
    print(e)

unsupported operand type(s) for -: 'INT_' and 'INT_'


### 파이썬 기본 클래스 확인하기 

#### None 키워드도 실제 클래스의 인스턴스를 보관한다

In [69]:
print(None.__class__)
print(None.__class__.__class__)
print(isinstance(None, None.__class__))
print(isinstance(None.__class__, type))

<class 'NoneType'>
<class 'type'>
True
True


#### None 값은 조건식으로 처리할 때 False로 인식

In [70]:
def func():
    return None

if func():
    print("True")
else:
    print("False")

False


#### NotImplemented도 클래스를 가지고 있다.

In [71]:
print(NotImplemented.__class__)
print(NotImplemented.__class__.__class__)
print(isinstance(NotImplemented, NotImplemented.__class__))
print(isinstance(NotImplemented.__class__, type))

<class 'NotImplementedType'>
<class 'type'>
True
True


#### 반환값으로 사용하면 True로 처리

In [72]:
def func(num=0):
    if num == 1:
        return NotImplemented
    else:
        return 0
    

if func(1):
    print("True")
else:
    print("False")

True


####  생략기호도 하나의 클래스이다.

In [73]:
Ellipsis
print(Ellipsis.__class__)
print(Ellipsis.__class__.__class__)
print(isinstance(Ellipsis, Ellipsis.__class__))
print(isinstance(Ellipsis.__class__, type))

<class 'ellipsis'>
<class 'type'>
True
True


#### 조건식 처리에서는 True 처리

In [74]:
def func(num=0):
    if num == 1:
        return Ellipsis
    else:
        return 0
    
if func(1):
    print("True")
else:
    print("False")

True


### 생략기호는 넘파이에서 검색할 때 사용도 가능하다

     차원에 대한 정보를 잘 모를 경우 생략기호를 사용하면 모든 것을 그대로 가져온다

In [75]:
import numpy as np

t = np.random.rand(2, 3, 4, 5)

# select 1st element from last dimension, copy rest
print(t[..., 0].shape)


# select 1st element from first dimension, copy rest
print(t[0, ...].shape)

(2, 3, 4)
(3, 4, 5)
